In [1]:
import yaml
import os
os.chdir('..')
print(os.getcwd())

z:\OMOP\omop_etl


In [2]:
from omop_etl.postproc.load import Loader, PRELOAD
load = Loader('config.yml')
# load.load_person()
# load.preload('measurement','height')

In [1]:
load.load_table('location')
# load.update_mappings('person')
# load.update_mappings('visit_occurrence')
# load.preload('observation')
# load.preload_all()

NameError: name 'load' is not defined

In [1]:
# %run -i run_etl.py

In [6]:
# truncate('preload','drug_exposure', load.engine)

In [7]:
f = open('config.yml')
load = yaml.safe_load(f)
# print(preload)
# list(preload['preload']['drug'].keys())

In [8]:
# STAGE = {
#     'person': 'PERSON',
#     'death': 'DEATH',
#     'condition_occurrence': 'CONDITION',
#     'procedure_occurrence': {
#         'cpt': 'PROCEDURE_CPT',
#         'icd': 'PROCEDURE_ICD'
#     },
#     'drug_exposure': {
#         'order': 'DRUG_ADMIN', 
#         'admin': 'DRUG_ORDER'
#     },
#     'measurement': {
#         'bp': 'MEASUREMENT_BP',
#         'res_pip': 'MEASUREMENT_Res_PIP',
#         'heart_rate': 'MEASUREMENT_HeartRate',
#         'height': 'MEASUREMENT_Height'
#     }
# }

# dp_list = []
# for t in load['load'].keys():
#     if load['load'][t]:
#         for part in load['load'][t].keys(): 
#             dp_list.append(STAGE[t][part])
#     else: 
#         dp_list.append(STAGE[t])
        
# print(dp_list)

In [9]:
from omop_etl.datastore import DataStore, execute
omop = DataStore('config.yml')
# t = omop.list_tables(in_schema=['preload'])

In [10]:
# omop.config_param['load']['condition_occurrence']
omop.find_column('location_id')

,schema_name,table_name,column_name
0,dbo,care_site,location_id
1,dbo,location,location_id
2,dbo,person,location_id


In [11]:
# for table in t.Table:
#     execute(f'drop table if exists preload.{table}', omop.engine)
sql_list = omop.get_bo_query('cohort_COVID_Broad')
sql_list.keys()

dict_keys(['CPT_CodingDetail', 'ICD_CodingDetail', 'Labs_UFHealth', 'Meds_by_name'])

In [12]:
sql_list['CPT_CodingDetail']

"SELECT  PX_ALL_PATIENTS.PATNT_KEY,  PX_ALL_PATIENTS.PATNT_ID,  PX_ALL_PATIENT_IDENTITIES.IDENT_ID_INT,  PX_JAX_MRN.IDENT_ID_INT FROM  ALL_PATIENTS PX_ALL_PATIENTS RIGHT OUTER JOIN ALL_PATIENT_SNAPSHOTS PX_ALL_PATIENT_SNAPSHOTS ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENTS.PATNT_KEY)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_ALL_PATIENT_IDENTITIES ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENT_IDENTITIES.PATNT_KEY and PX_ALL_PATIENT_IDENTITIES.LOOKUP_IND = 'Y' and PX_ALL_PATIENT_IDENTITIES.IDENT_ID_TYPE = 101)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_JAX_MRN ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_JAX_MRN.PATNT_KEY and PX_JAX_MRN.LOOKUP_IND = 'Y' and PX_JAX_MRN.IDENT_ID_TYPE = 110)  RIGHT OUTER JOIN PROCEDURE_EVENT_DTL ON (PROCEDURE_EVENT_DTL.PATNT_SNAPSHT_KEY=PX_ALL_PATIENT_SNAPSHOTS.PATNT_SNAPSHT_KEY)  LEFT OUTER JOIN ALL_CPT_PROCEDURE_CODES ON (PROCEDURE_EVENT_DTL.CPT_CD_KEY=ALL_CPT_PROCEDURE_CODES.CPT_CD_KEY)  WHERE  (  PROCEDURE_EVENT_DTL.START_DATE >= '01/01/

In [13]:
import sqlparse

sql_list = omop.get_bo_query('cohort_COVID_Broad')
dp_name = 'Meds_by_name'

sql_string = sql_list[dp_name]
sql_parsed = sqlparse.parse(sql_string)
tks = sql_parsed[0]
idx = [tks.token_index(t) for t in tks if t.is_keyword and t.value == 'FROM'][0]
id_list =sql_parsed[0].token_prev(idx)[1]

colnames = [i.value.split('.')[-1] for i in id_list]
dup_cols = set([x for x in colnames if colnames.count(x) > 1])

new_colnames = []
for i,item in enumerate(id_list):
    if isinstance(item, sqlparse.sql.Identifier):
        colname = item.value.split('.')[-1]
        tabname = item.value.split('.')[-2]
        tabname = '_'.join([w[:3] for w in tabname.split('_')])
        if colname in dup_cols:
            item.value = f'{item.value} AS {tabname}_{colname}'
            # print(item.value)
        new_colnames.append(item.value)

colnames_str = ', '.join(new_colnames)

into_str = f'{colnames_str} INTO {dp_name}'
id_list.value = into_str
sql_string = ''.join([t.value for t in tks])
# sql_string = sql_string.replace(id_list.value, into_str).replace("'","''")
# print(sqlparse.format(sql_string, reindent_aligned=True, comma_first=True))
print(sql_string)

  dbo.ALL_PATIENT_IDENTITIES.IDENT_ID_TYPE = 101  ) Table__1308 ON (Table__1308.PATNT_KEY=ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_KEY)  LEFT OUTER JOIN dbo.MED_ORDER_MIX_DTL ON (dbo.MED_ORDER_DTL.MED_ORDR_KEY=dbo.MED_ORDER_MIX_DTL.MED_ORDR_KEY)  LEFT OUTER JOIN dbo.ALL_MEDICATIONS ALL_MEDICATIONS1 ON (dbo.MED_ORDER_MIX_DTL.MED_CD_KEY=ALL_MEDICATIONS1.MED_CD_KEY)  LEFT OUTER JOIN dbo.MED_DISPENSE_DTL ON (dbo.MED_ORDER_MIX_DTL.MED_ORDR_KEY=dbo.MED_DISPENSE_DTL.MED_ORDR_KEY)  WHERE  (  (   Table__1308.IDENT_ID_INT Is Not Null    OR   Table__1117.IDENT_ID_INT Is Not Null   )  AND  (   dbo.MED_ADMINISTRATION_DTL.TAKN_DT >= '01/01/2020 00:0:0'   OR   dbo.MED_ORDER_DTL.ORDR_DATE >= '01/01/2020 00:0:0'   OR   dbo.MED_DISPENSE_DTL.ACTION_INSTANT_DT >= '01/01/2020 00:0:0'  )  AND  ALL_MEDICATIONS1.MED_NAME LIKE '%REMDESIVIR%'  AND  ( dbo.ALL_PATIENTS.TEST_IND='N' )  )


In [14]:
from omop_etl.cohort import format_cohort_sql, stage_cohort
sql_list = omop.get_bo_query('cohort_COVID_Broad')
dp_name = 'Meds_by_name'
sql_string = format_cohort_sql(sql_list[dp_name], dp_name)
print(sqlparse.format(sql_string, reindent_aligned=True, comma_first=True))

DROP TABLE IF EXISTS DWS_OMOP.cohort.Meds_by_name;SELECT dbo.ALL_PATIENTS.PATNT_KEY,
       dbo.ALL_PATIENTS.PATNT_ID,
       Table__1308.IDENT_ID_INT AS Tab__130_IDENT_ID_INT,
       Table__1117.IDENT_ID_INT AS Tab__111_IDENT_ID_INT INTO DWS_OMOP.cohort.Meds_by_name
  FROM dbo.MED_ADMINISTRATION_DTL
 RIGHT OUTER JOIN dbo.MED_ORDER_DTL
    ON (dbo.MED_ADMINISTRATION_DTL.MED_ORDR_KEY=dbo.MED_ORDER_DTL.MED_ORDR_KEY AND dbo.MED_ADMINISTRATION_DTL.LINK_ORDR_IND<>''Y'')
 RIGHT OUTER JOIN dbo.PATIENT_ENCOUNTER_DTL
    ON (dbo.PATIENT_ENCOUNTER_DTL.PATNT_ENCNTR_KEY=dbo.MED_ORDER_DTL.LINK_PATNT_ENCNTR_KEY AND dbo.PATIENT_ENCOUNTER_DTL.TEST_IND=''N'')
  LEFT OUTER JOIN dbo.ALL_PATIENT_SNAPSHOTS ALL_PT_SNAPSHOTS_ENCOUNTER
    ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_SNAPSHT_KEY=dbo.PATIENT_ENCOUNTER_DTL.DSCHRG_PATNT_SNAPSHT_KEY AND dbo.PATIENT_ENCOUNTER_DTL.TEST_IND=''N'')
  LEFT OUTER JOIN dbo.ALL_PATIENTS
    ON (ALL_PT_SNAPSHOTS_ENCOUNTER.PATNT_KEY=dbo.ALL_PATIENTS.PATNT_KEY)
  LEFT OUTER JOIN (


In [12]:
# ', '.join(new_colnames)
# items
print(sqlparse.format(stage_cohort(sql_list), reindent_aligned=True, comma_first=True))

INSERT INTO cohort.PersonList WITH (TABLOCK) SELECT DISTINCT *
  FROM (
        select patnt_key
          from cohort.CPT_CodingDetail
         union select patnt_key
          from cohort.ICD_CodingDetail
         union select patnt_key
          from cohort.Labs_UFHealth
         union select patnt_key
          from cohort.Meds_by_name
       ) x


In [15]:
import sqlparse
from omop_etl.datastore import DataStore, execute
from omop_etl.cohort import format_cohort_sql

omop = DataStore('config.yml')
sql_list = omop.get_bo_query('cohort_COVID_Broad')

test_list = list(sql_list.keys())
for t in test_list:
    if len(sql_list[t]) == 0: pass
    else: execute("execute ('use [DWS_PROD]; {}')".format(format_cohort_sql(sql_list[t], t)), omop.engine)

In [16]:
# add_into_clause(sql_list['CPT_Professional'], 'CPT_Professional')
execute(stage_cohort(sql_list), omop.engine)

'226328 rows affected'

In [21]:
# stack_parsed

In [124]:
# tks.token_first()
# for t in tks.flatten():
#     print(t)

%run -i omop_etl/pullrawdata/pull_raw_data_v2.py

In [1]:
a = False
if a == False:
    print('a is False')

a is False
